In [1]:
# ============================================================================
# STEP 1: ENVIRONMENT SETUP
# ============================================================================
# Load environment variables and initialize Google Earth Engine

from dotenv import load_dotenv
load_dotenv()

import ee, eemont
from forestry_carbon_arr.core import ForestryCarbonARR
from forestry_carbon_arr.utils.zarr_utils import save_dataset_efficient_zarr, load_dataset_zarr

import gcsfs
import os

fs = gcsfs.GCSFileSystem(project=os.getenv("GOOGLE_CLOUD_PROJECT"), token='/usr/src/app/user_id.json')

forestry = ForestryCarbonARR(config_path='./00_input/korindo.json')
forestry.initialize_gee()

✓ GEE Initialized successfully
  Credentials Path: /usr/src/app/user_id.json - loaded successfully


In [2]:
# ============================================================================
# STEP 2: LOAD AREA OF INTEREST (AOI) but only on the AOI concession
# ============================================================================
# Load the area of interest geometry for reference

from forestry_carbon_arr.core.utils import DataUtils
import geopandas as gpd
import geemap

data_utils = DataUtils(forestry.config, use_gee=True)
aoi_gpd, aoi_ee = data_utils.load_geodataframe_gee('./00_input/korindo_buffer.shp')

aoi_gpd_utm = aoi_gpd.to_crs(epsg=32749)

print(f"✅ AOI loaded: {len(aoi_gpd_utm)} features")
print(f"   Area: {aoi_gpd_utm.geometry.area.sum()/10000:.2f} hectares")

✅ AOI loaded: 1 features
   Area: 40740.66 hectares


In [3]:
## save data
zarr_path = 'gs://remote_sensing_saas/01-korindo/tsfresh_model_catboost/ds_features_all_aoi.zarr'

ds_ml_aoi = load_dataset_zarr(
    zarr_path,
)
ds_ml_aoi

📂 Loading dataset from GCS zarr: gs://remote_sensing_saas/01-korindo/tsfresh_model_catboost/ds_features_all_aoi.zarr
✅ Dataset loaded: {'sample': 4073658, 'features': 197}


/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:855: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"✅ Dataset loaded: {dict(ds.dims)}")


<xarray.Dataset> Size: 7GB
Dimensions:     (sample: 4073658, features: 197)
Coordinates:
  * features    (features) <U68 54kB 'EVI__fft_coefficient__attr_"angle"__coe...
  * sample      (sample) int64 33MB 0 1 2 3 ... 4073654 4073655 4073656 4073657
Data variables:
    coord_x     (sample) float64 33MB dask.array<chunksize=(100000,), meta=np.ndarray>
    coord_y     (sample) float64 33MB dask.array<chunksize=(100000,), meta=np.ndarray>
    plot_id     (sample) object 33MB dask.array<chunksize=(509208,), meta=np.ndarray>
    X_features  (sample, features) float64 6GB dask.array<chunksize=(100000, 197), meta=np.ndarray>

In [4]:
import os
import json
from google.cloud import storage
from catboost import CatBoostClassifier

use_output_prediction = True

model = 'gs://remote_sensing_saas/01-korindo/tsfresh_model_catboost/catboost_20251204_041716.cbm'
config = 'gs://remote_sensing_saas/01-korindo/tsfresh_model_catboost/config_20251204_041716.json'

zarr_path = 'gs://remote_sensing_saas/01-korindo/tsfresh_model_catboost/ds_pred_aoi.zarr'

def download_blob(gcs_uri, local_filename):
    """Downloads a blob from the bucket."""
    storage_client = storage.Client()
    
    # Parse the gs:// URI
    parts = gcs_uri.replace("gs://", "").split("/", 1)
    bucket_name = parts[0]
    blob_name = parts[1]
    
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.download_to_filename(local_filename)
    print(f"Downloaded {gcs_uri} to {local_filename}")

if use_output_prediction != True:

    # 2. Download files locally
    # CatBoost needs a local file to load the model efficiently
    local_model_file = '/mnt/data/catboost_20251204_041716.cbm'
    local_config_file = '/mnt/data/config_20251204_041716.json'

    download_blob(model, local_model_file)
    download_blob(config, local_config_file)

    # 3. Load the Config (JSON)
    with open(local_config_file, 'r') as f:
        config_data = json.load(f)
        print("Config loaded.")

    # 4. Load the CatBoost Model
    # Note: Ensure you use the same class (Classifier/Regressor) used during training
    model = CatBoostClassifier() # Change to CatBoostRegressor() if your target is continuous
    model.load_model(local_model_file)

    # 5. Prepare Data and Predict
    # Assuming ds_ml_aoi is already defined in your environment
    X_all = ds_ml_aoi.X_features.values

    # CRITICAL FIX: You predict using the 'model' object, not the 'config'
    y_pred = model.predict(X_all)

    print("Predictions shape:", y_pred.shape)

    ds_ml_aoi['y_pred'] = (('samples'), y_pred)
    # ds_ml_aoi
    # save first before cleaning
    
    chunk = {'samples':100000}
    save_dataset_efficient_zarr(ds_ml_aoi, zarr_path, chunk_sizes=chunk, overwrite=True)

else:
    ds_ml_aoi = load_dataset_zarr(zarr_path)
    # ds_ml_aoi



📂 Loading dataset from GCS zarr: gs://remote_sensing_saas/01-korindo/tsfresh_model_catboost/ds_pred_aoi.zarr
✅ Dataset loaded: {'sample': 4073658, 'features': 197, 'samples': 4073658}


/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:855: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"✅ Dataset loaded: {dict(ds.dims)}")


In [5]:
ds_ml_aoi

<xarray.Dataset> Size: 7GB
Dimensions:     (sample: 4073658, features: 197, samples: 4073658)
Coordinates:
  * features    (features) <U68 54kB 'EVI__fft_coefficient__attr_"angle"__coe...
  * sample      (sample) int64 33MB 0 1 2 3 ... 4073654 4073655 4073656 4073657
Dimensions without coordinates: samples
Data variables:
    coord_x     (sample) float64 33MB dask.array<chunksize=(100000,), meta=np.ndarray>
    coord_y     (sample) float64 33MB dask.array<chunksize=(100000,), meta=np.ndarray>
    plot_id     (sample) object 33MB dask.array<chunksize=(509208,), meta=np.ndarray>
    X_features  (sample, features) float64 6GB dask.array<chunksize=(100000, 197), meta=np.ndarray>
    y_pred      (samples) int64 33MB dask.array<chunksize=(100000,), meta=np.ndarray>

In [6]:
# ============================================================================
# POST-PROCESSING: REMOVE SMALL OBJECTS FROM PREDICTIONS
# ============================================================================
# Remove small isolated patches (< 0.25 ha) and merge with surrounding pixels

def remove_small_objects_2d(pred_array, min_size_pixels, connectivity=1):
    """
    Remove small objects from 2D prediction array
    
    Parameters:
    -----------
    pred_array : np.ndarray
        2D array with predictions (0 or 1, can have NaN)
    min_size_pixels : int
        Minimum size in pixels to keep
    connectivity : int
        Connectivity for connected components (1=4-connected, 2=8-connected)
        
    Returns:
    --------
    cleaned_array : np.ndarray
        Array with small objects removed (merged with surrounding majority class)
    """
    from skimage.morphology import remove_small_objects
    from scipy.ndimage import generic_filter
    
    # Create a copy
    cleaned = pred_array.copy()
    
    # Handle NaN values - use mode of surrounding pixels
    nan_mask = np.isnan(cleaned)
    if np.any(nan_mask):
        # Fill NaN with mode of non-NaN neighbors
        def mode_filter(values):
            vals = values[~np.isnan(values)]
            if len(vals) == 0:
                return np.nan
            return np.bincount(vals.astype(int)).argmax()
        
        # Apply filter to entire array, then use only where we have NaN
        filtered = generic_filter(cleaned, mode_filter, size=3, mode='constant', cval=np.nan)
        # Only fill NaN positions with filtered values
        cleaned[nan_mask] = filtered[nan_mask]
    
    # Convert to binary (0 or 1) for each class
    # Process eligible class (1)
    eligible_mask = (cleaned == 1).astype(bool)
    if np.any(eligible_mask):
        # Remove small eligible objects
        eligible_cleaned = remove_small_objects(eligible_mask, min_size=min_size_pixels, connectivity=connectivity)
        # Update: keep only large eligible objects, rest become ineligible
        cleaned[eligible_cleaned] = 1
        cleaned[eligible_mask & ~eligible_cleaned] = 0
    
    # Process ineligible class (0) - remove small ineligible objects
    ineligible_mask = (cleaned == 0).astype(bool)
    if np.any(ineligible_mask):
        # Remove small ineligible objects
        ineligible_cleaned = remove_small_objects(ineligible_mask, min_size=min_size_pixels, connectivity=connectivity)
        # Update: keep only large ineligible objects, rest become eligible (merge with surrounding)
        cleaned[ineligible_cleaned] = 0
        cleaned[ineligible_mask & ~ineligible_cleaned] = 1
    
    # Restore NaN values
    cleaned[nan_mask] = np.nan
    
    return cleaned

# Calculate pixel size from coordinate spacing
print("\n🔧 Post-processing: Removing small objects (< 0.25 ha)")
print("=" * 60)


🔧 Post-processing: Removing small objects (< 0.25 ha)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
import xarray as xr

# =========================================================
# 1. SETUP & PIXEL CALCULATION
# =========================================================

ds_ml_pred = ds_ml_aoi
ds_ml_pred['eligibility_pred'] = (('sample'), ds_ml_pred['y_pred'].data)
eligibility_pred = ds_ml_pred['eligibility_pred'].data

# Compute plot_id values
if hasattr(ds_ml_pred['plot_id'].values, 'compute'):
    plot_ids = ds_ml_pred['plot_id'].values.compute()
else:
    plot_ids = ds_ml_pred['plot_id'].values

unique_plot_ids = np.unique(plot_ids)

# Calculate pixel size from sample
sample_plot_id = unique_plot_ids[0]
plot_mask_sample = plot_ids == sample_plot_id
x_coords_sample = ds_ml_pred['coord_x'].isel(sample=plot_mask_sample).values
y_coords_sample = ds_ml_pred['coord_y'].isel(sample=plot_mask_sample).values

if hasattr(x_coords_sample, 'compute'):
    x_coords_sample = x_coords_sample.compute()
    y_coords_sample = y_coords_sample.compute()

x_unique_sample = np.unique(x_coords_sample)
y_unique_sample = np.unique(y_coords_sample)

# Default values
min_size_pixels = 25 # Fallback
min_area_ha = 0.25

if len(x_unique_sample) > 1 and len(y_unique_sample) > 1:
    pixel_size_x = np.abs(x_unique_sample[1] - x_unique_sample[0])
    pixel_size_y = np.abs(y_unique_sample[1] - y_unique_sample[0])
    pixel_area_ha = (pixel_size_x * pixel_size_y) / 10000
    
    # Calculate threshold
    min_size_pixels = int(np.ceil(min_area_ha / pixel_area_ha))
    
    print(f"📏 Pixel Config: {pixel_size_x:.1f}m x {pixel_size_y:.1f}m")
    print(f"📏 Min Area: {min_area_ha} ha = {min_size_pixels} pixels")

# =========================================================
# 2. POST-PROCESSING (CLEANING)
# =========================================================
print(f"\n🧹 Applying post-processing (removing < {min_area_ha} ha)...")

eligibility_pred_cleaned = eligibility_pred.copy()

for plot_id_value in unique_plot_ids:
    plot_mask = plot_ids == plot_id_value
    
    # Extract data
    if hasattr(ds_ml_pred['eligibility_pred'].values, 'compute'):
        plot_pred = ds_ml_pred['eligibility_pred'].isel(sample=plot_mask).values.compute()
        x_coords = ds_ml_pred['coord_x'].isel(sample=plot_mask).values.compute()
        y_coords = ds_ml_pred['coord_y'].isel(sample=plot_mask).values.compute()
    else:
        plot_pred = ds_ml_pred['eligibility_pred'].isel(sample=plot_mask).values
        x_coords = ds_ml_pred['coord_x'].isel(sample=plot_mask).values
        y_coords = ds_ml_pred['coord_y'].isel(sample=plot_mask).values
    
    # Grid creation
    x_unique = np.unique(x_coords)
    y_unique = np.unique(y_coords)
    x_to_idx = {float(x): i for i, x in enumerate(x_unique)}
    y_to_idx = {float(y): i for i, y in enumerate(y_unique)}
    
    pred_grid = np.full((len(y_unique), len(x_unique)), np.nan)
    
    # Fill grid
    for i in range(len(x_coords)):
        x_idx = x_to_idx.get(float(x_coords[i]))
        y_idx = y_to_idx.get(float(y_coords[i]))
        if x_idx is not None and y_idx is not None:
            pred_grid[y_idx, x_idx] = plot_pred[i]
            
    # Apply cleaning function (Assumes remove_small_objects_2d is defined)
    pred_grid_cleaned = remove_small_objects_2d(pred_grid, min_size_pixels, connectivity=2)
    
    # Map back to 1D
    plot_indices = np.where(plot_mask)[0]
    for i, idx in enumerate(plot_indices):
        x_idx = x_to_idx.get(float(x_coords[i]))
        y_idx = y_to_idx.get(float(y_coords[i]))
        if x_idx is not None and y_idx is not None:
            eligibility_pred_cleaned[idx] = pred_grid_cleaned[y_idx, x_idx]

ds_ml_pred['eligibility_pred_cleaned'] = (('sample',), eligibility_pred_cleaned)
print("✅ Post-processing completed!")

# =========================================================
# 3. SINGLE MAP VISUALIZATION (PREDICTION ONLY)
# =========================================================
print("\n🗺️  Visualizing Cleaned Predictions per Plot")
print("=" * 60)

for plot_id_value in unique_plot_ids:
    plot_mask = plot_ids == plot_id_value
    
    # 1. Get Data (Cleaned Prediction)
    if hasattr(ds_ml_pred['eligibility_pred_cleaned'].values, 'compute'):
        pred_data = ds_ml_pred['eligibility_pred_cleaned'].isel(sample=plot_mask).values.compute()
        x_coords = ds_ml_pred['coord_x'].isel(sample=plot_mask).values.compute()
        y_coords = ds_ml_pred['coord_y'].isel(sample=plot_mask).values.compute()
    else:
        pred_data = ds_ml_pred['eligibility_pred_cleaned'].isel(sample=plot_mask).values
        x_coords = ds_ml_pred['coord_x'].isel(sample=plot_mask).values
        y_coords = ds_ml_pred['coord_y'].isel(sample=plot_mask).values

    # Skip if no valid data
    if np.all(np.isnan(pred_data)):
        print(f"⚠️ Plot {plot_id_value}: No valid predictions.")
        continue

    # 2. Create Grid
    x_unique = np.unique(x_coords)
    y_unique = np.unique(y_coords)
    x_to_idx = {float(x): i for i, x in enumerate(x_unique)}
    y_to_idx = {float(y): i for i, y in enumerate(y_unique)}
    
    pred_grid = np.full((len(y_unique), len(x_unique)), np.nan)
    
    for i in range(len(x_coords)):
        x_idx = x_to_idx.get(float(x_coords[i]))
        y_idx = y_to_idx.get(float(y_coords[i]))
        if x_idx is not None and y_idx is not None:
             pred_grid[y_idx, x_idx] = pred_data[i]

    # 3. Plotting
    # Create a single figure (no subplots)
    plt.figure(figsize=(10, 8))
    
    # Define custom colormap (Red=0, Green=1)
    colors = ['#ff6b6b', '#51cf66'] # Red, Green
    cmap = ListedColormap(colors)
    
    # Plot Image
    # 'nearest' ensures sharp pixel boundaries
    # origin='lower' is standard for geographic coordinates (y increases upwards)
    im = plt.imshow(pred_grid, cmap=cmap, vmin=0, vmax=1, 
                    origin='lower', interpolation='nearest',
                    extent=[x_unique.min(), x_unique.max(), y_unique.min(), y_unique.max()])

    # Add Custom Legend
    patches = [
        mpatches.Patch(color=colors[0], label='Ineligible'),
        mpatches.Patch(color=colors[1], label='Eligible')
    ]
    plt.legend(handles=patches, loc='upper right', frameon=True, fontsize=11)

    # Styling
    plt.title(f"Predicted Eligibility: Plot {plot_id_value}\n(Cleaned: min area {min_area_ha} ha)", 
              fontsize=14, fontweight='bold', pad=15)
    plt.xlabel('X Coordinate (m)')
    plt.ylabel('Y Coordinate (m)')
    
    # Calculate stats for title/console
    eligible_px = np.nansum(pred_grid == 1)
    ineligible_px = np.nansum(pred_grid == 0)
    total_px = eligible_px + ineligible_px
    eligible_ha = eligible_px * pixel_area_ha
    
    # Add stats box
    stats_text = (f"Eligible Area: {eligible_ha:.2f} ha\n"
                  f"Eligible Pixels: {int(eligible_px):,}\n"
                  f"Coverage: {eligible_px/total_px:.1%}")
    
    plt.text(0.02, 0.98, stats_text, transform=plt.gca().transAxes,
             fontsize=10, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

    plt.tight_layout()
    plt.show()

    print(f"Plot {plot_id_value} visualized. Eligible Area: {eligible_ha:.2f} ha")

📏 Pixel Config: 10.0m x 10.0m
📏 Min Area: 0.25 ha = 25 pixels

🧹 Applying post-processing (removing < 0.25 ha)...
